In [1]:
import cv2
import subprocess
import os
import numpy as np
from os.path import isfile, join
import natsort
import re
import tensorflow as tf
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import glob
from sklearn.model_selection import train_test_split
from skimage.io import imread
from skimage.transform import pyramid_reduce, resize

In [2]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [3]:
!mkdir frames

In [12]:
def video2frame(invideofilename, save_path):
    vidcap = cv2.VideoCapture(invideofilename)
    count = 0
    while True:
      success,image = vidcap.read()
      if not success:
          break
      fname = "{}.jpg".format("{0:05d}".format(count))
      cv2.imwrite(save_path + fname, image)
      count += 1

video2frame('star.mp4','frames/')

In [13]:
data_list = sorted(glob.glob('frames/*.jpg'))
IMG_SIZE = 512
model = tf.keras.models.load_model('drive/My Drive/model/first_model')

In [14]:
img_data = np.empty((len(data_list),512,512,3),dtype=np.float)
for i, img_path in enumerate(data_list):
  img = imread(img_path)
  img_res = resize(img, output_shape=(IMG_SIZE, IMG_SIZE, 3))
  img_data[i]=img_res
img_data= np.array(img_data,dtype=np.double)
np.save('img_data.npy',img_data)

In [16]:
preds = model.predict(img_data)

In [32]:
mask_data = np.empty((len(preds),512,512,1),dtype=np.int8)
for i, pred in enumerate(preds):
  pred[pred < 0.05] = 0
  pred[pred >= 0.05] = 255
  mask_data[i]=pred
mask_data= np.array(mask_data,dtype=np.uint8)

In [18]:
!mkdir frames_output

In [30]:
def inpaint(img,mask):
    img_cp = img.copy()
    res = cv2.inpaint(src=img_cp, inpaintMask=mask,
                     inpaintRadius=3, flags=cv2.INPAINT_TELEA)
    return res

for i in range(len(data_list)):
    img = cv2.imread(data_list[i], cv2.IMREAD_COLOR)
    img_siz = img.shape
    img_res = resize(img, output_shape=(IMG_SIZE, IMG_SIZE, 3))
    img_res *= 255
    img_dt= np.array(img_res,dtype = np.uint8)
    mask = mask_data[i].squeeze()
    res = inpaint(img_dt,mask)
    res_res = resize(res, output_shape=img_siz)
    res_res *= 255
    cv2.imwrite('frames_output/'+data_list[i][7:12]+'.jpg',res_res)

In [31]:
 def solution(files):
    file_lst = [re.split('([0-9]+)',i) for i in files]
    file_lst.sort( key = lambda x : ( x[0].lower(), int(x[1]) ) )

    return [''.join(lst) for lst in file_lst]

pathIn = 'frames_output/'
pathOut = 'star_output.mp4'
fps = 24
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
files.sort(key = lambda x: x[5:-4])
files.sort()

frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
files = solution(files)

for i in range(len(files)):
    filename=pathIn + '/' + files[i]
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)

    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
    out.write(frame_array[i])
out.release()